# Climatic data

https://www.renewables.ninja/
Country/Weather data weighted by population.



In [1]:
import pandas as pd
from project.thermal import TEMP_BASE, ORIENTATION_FACTOR

In [2]:
data = pd.read_csv('ninja_weather_country_FR_merra-2_population_weighted.csv', skiprows=[0, 1], index_col=[0], parse_dates=[0])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 350640 entries, 1980-01-01 00:00:00 to 2019-12-31 23:00:00
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   precipitation       350640 non-null  float64
 1   temperature         350640 non-null  float64
 2   irradiance_surface  350640 non-null  float64
 3   irradiance_toa      350640 non-null  float64
 4   snowfall            350640 non-null  float64
 5   snow_mass           350640 non-null  float64
 6   cloud_cover         350640 non-null  float64
 7   air_density         350640 non-null  float64
dtypes: float64(8)
memory usage: 24.1 MB


In [4]:
variables = ['temperature', 'irradiance_surface']

## Calculation

In [5]:
d_hs_yr, temp_hs_yr, irr_hs_yr = dict(), dict(), dict()
d_hs_month, temp_hs_month, irr_hs_month = dict(), dict(), dict()
d_hs_day, temp_hs_day, irr_hs_day = dict(), dict(), dict()

for year in data.index.year.unique():

    d = data.loc[data.index.year == year, variables]
    d = d.groupby([d.index.month, d.index.day]).agg({'temperature': 'mean', 'irradiance_surface': 'sum'})
    d.index.names = ['month', 'day']

    hs = d['temperature'] < TEMP_BASE

    # yearly data
    d_hs_yr.update({year: hs.sum()})
    temp_hs_yr.update({year: (hs * d['temperature']).sum() / d_hs_yr[year]})
    irr_hs_yr.update({year:  (hs * d['irradiance_surface']).sum() / 1000})

    # monthly data
    d_hs_month.update({year: hs.groupby('month').sum()})
    temp_hs_month.update({year: (hs * d['temperature']).groupby('month').sum() / d_hs_month[year]})
    irr_hs_month.update({year: (hs * d['irradiance_surface']).groupby('month').sum() / 1000})

    # dayly data
    d_hs_day.update({year: hs.groupby(['month', 'day']).sum()})
    temp_hs_day.update({year: (hs * d['temperature']).groupby(['month', 'day']).sum() / d_hs_month[year]})
    irr_hs_day.update({year: (hs * d['irradiance_surface']).groupby(['month', 'day']).sum() / 1000})

result_year = pd.concat((pd.Series(d_hs_yr), pd.Series(temp_hs_yr), pd.Series(irr_hs_yr)), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])
result_month = pd.concat((pd.DataFrame(d_hs_month).rename_axis('year', axis=1).stack(), pd.DataFrame(temp_hs_month).rename_axis('year', axis=1).stack(), pd.DataFrame(irr_hs_month).rename_axis('year', axis=1).stack()), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])
result_day = pd.concat((pd.DataFrame(d_hs_day).rename_axis('year', axis=1).stack(), pd.DataFrame(temp_hs_day).rename_axis('year', axis=1).stack(), pd.DataFrame(irr_hs_day).rename_axis('year', axis=1).stack()), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])

In [6]:
var = 'irr_hs (kWh/m2)'
for key, factor in ORIENTATION_FACTOR.items():
    result_year['{} {}'.format(var, key)] = result_year[var] * factor
    result_month['{} {}'.format(var, key)] = result_month[var] * factor
    result_day['{} {}'.format(var, key)] = result_day[var] * factor

In [7]:
# exporting data yearly
variables = ['d_hs', 'temp_hs', 'irr_hs (kWh/m2) Mean']
export_resirf = result_year[variables]
export_resirf.columns = ['DAYS_HEATING_SEASON', 'TEMP_EXT', 'SOLAR_RADIATION']
export_resirf.to_csv('output/climatic_data.csv')

In [8]:
result_year.mean()

d_hs                     199.600000
temp_hs                    5.952009
irr_hs (kWh/m2)          501.011548
irr_hs (kWh/m2) South    551.112703
irr_hs (kWh/m2) West     285.576583
irr_hs (kWh/m2) Est      285.576583
irr_hs (kWh/m2) North    100.202310
irr_hs (kWh/m2) Mean     305.617044
dtype: float64

In [9]:
years = (2014, 2015, 2016)

In [10]:
export_resirf_daily = result_day.xs(2006, level='year').loc[:, variables].fillna(0)
export_resirf_daily.columns = ['DAYS_HEATING_SEASON', 'TEMP_EXT', 'SOLAR_RADIATION']
export_resirf_daily.to_csv('output/climatic_data_daily.csv')